In [2]:
import pandas as pd
import numpy as np
import keras
import gensim
import pickle
import gc
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
%matplotlib inline

%load_ext autoreload
%autoreload 2
import helper

Using TensorFlow backend.


In [3]:
X_train = np.load("data/word_vectors/pubmed_stratify_no_wiki_X_train_seeds42.npy")
y_train = np.load("data/word_vectors/pubmed_stratify_no_wiki_y_train_seeds42.npy")

In [4]:
X_test = np.load("data/word_vectors/pubmed_stratify_no_wiki_X_test_seeds42.npy")
y_test = np.load("data/word_vectors/pubmed_stratify_no_wiki_y_test_seeds42.npy")

In [5]:
X_train.shape

(41028, 100, 200)

In [5]:
input_shape = (100, 200)

# Basic RNN (LSTM)

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.LSTM(100, input_shape=input_shape))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [15]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 23)                2323      
Total params: 122,723
Trainable params: 122,723
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 20s - loss: 2.5846 - categorical_accuracy: 0.2428 - val_loss: 2.3985 - val_categorical_accuracy: 0.2959
Epoch 2/15
36925/36925 [==============================] - 19s - loss: 2.3931 - categorical_accuracy: 0.2935 - val_loss: 2.4358 - val_categorical_accuracy: 0.2915
Epoch 3/15
36925/36925

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0

# Basic RNN (GRU)

In [16]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.GRU(100, input_shape=input_shape))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [17]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 100)               90300     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 23)                2323      
Total params: 92,623
Trainable params: 92,623
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 16s - loss: 2.3993 - acc: 0.2890 - val_loss: 1.9877 - val_acc: 0.3907
Epoch 2/15
36925/36925 [==============================] - 16s - loss: 1.8964 - acc: 0.3986 - val_loss: 1.7821 - val_acc: 0.4353
Epoch 3/15
36925/36925 [==============================] - 16s - loss: 1.7657 - acc: 0.4277 -

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


98

# Deep RNN (GRU)

In [20]:
model = keras.models.Sequential()
model.add(keras.layers.recurrent.GRU(128, input_shape=input_shape, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.recurrent.GRU(64))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [21]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 100, 128)          126336    
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
gru_8 (GRU)                  (None, 64)                37056     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 23)                1495      
Total params: 164,887
Trainable params: 164,887
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 28s - loss: 2.4373 - cat

0

# Basic CNN

In [22]:
filters = 100

model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size=1,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.pooling.GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(23, activation='softmax'))

In [23]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 100, 100)          20100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 23)                2323      
Total params: 22,423
Trainable params: 22,423
Non-trainable params: 0
_________________________________________________________________
None
Train on 36925 samples, validate on 4103 samples
Epoch 1/15
36925/36925 [==============================] - 5s - loss: 2.1421 - categorical_accuracy: 0.3481 - val_loss: 1.8721 - val_categorical_accuracy: 0.4143
Epoch 2/15
36925/36925 [==============================]

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


102

# Basic Recurrent CNN

In [24]:
input_shape = (100, 200)
filters = 128
kernel_size = 3
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.MaxPool1D(pool_size=16))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.recurrent.GRU(64))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(23, activation='softmax'))

In [25]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 98, 128)           76928     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 6, 128)            0         
_________________________________________________________________
gru_9 (GRU)                  (None, 64)                37056     
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 23)                1495      
Total params: 115,479
Trainable params: 115,479
Non-trainable params: 0
_________________________________________________________________
None

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


100

# Deep Recurrent CNN

In [26]:
input_shape = (100, 200)
filters = 128
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(filters,
             kernel_size=2,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.MaxPool1D(pool_size=16))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv1D(filters,
             kernel_size=3,
             padding='valid',
             activation='relu',
             strides=1, input_shape=input_shape))
model.add(keras.layers.MaxPool1D(pool_size=2))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.recurrent.GRU(64))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(23, activation='softmax'))

In [27]:
model.compile(optimizer="adagrad",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_split=0.1, epochs=15, batch_size=256)
yhat = model.predict(X_test)
_ = helper.score_prediction(y_test, yhat, binary=False)
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 99, 128)           51328     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 6, 128)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 4, 128)            49280     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 128)            0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 2, 128)            0         
_________________________________________________________________
gru_10 (GRU)                 (None, 64)                37056     
__________

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


104